In [ ]:
import sys
# !{sys.executable} -m pip install coolprop -U
# !{sys.executable} -m pip install handcalcs
# !{sys.executable} -m pip install pint
#from IPython.display import display, update_display

#Definition function for pressure drop
import math
from CoolProp.CoolProp import PropsSI
import sympy as sp
from sympy import symbols, Eq, solve,S,N,Float
sp.init_printing()
from handcalcs import *  #.render
from handcalcs import render
from pint import UnitRegistry
u = UnitRegistry()
Q_ = u.Quantity

def Colebrook(Re,K_):
    """ caculation   
    """
    X1 = K_ * Re * 0.123968186335418
    X2 = math.log(Re) - 0.779397488455682
    F = X2 - 0.2
    e = (math.log(X1 + F) + F - X2) / (1 + X1 + F)
    F = F - (1 + X1 + F + 0.5 * e) * e * (X1 + F) / (1 + X1 + F + e * (1 + e / 3))
    e = (math.log(X1 + F) + F - X2) / (1 + X1 + F)
    F = F - (1 + X1 + F + 0.5 * e) * e * (X1 + F) / (1 + X1 + F + e * (1 + e / 3))
    F = 1.15129254649702 / F
    Colebrook = F * F
    return Colebrook
def Re_(DN_mm , v_m_s , rho_g_l , mu_Pas ):
    Re_ = (1 / 1000) * (DN_mm * v_m_s * rho_g_l) / mu_Pas
    return Re_
def Lambda_dp(DN_mm,Re):
    K_ = 0.05 / DN_mm
    if Re <= 2300:
        Lambda_dp = 64 / Re #poiseuille
    if Re > 2300 and Re < 100000:
        Lambda_dp = 0.3164 * Re ^ (-0.25) #"blasius
    if Re >= 100000:
        Lambda_dp = Colebrook(Re, K_)
    #
    #cf TM1 GRETH 2.3.b)
    #=0,25*(LOG(13/E333+(0,05/E339)/3,7))^-2
    #Cf = 0.25 * (WorksheetFunction.Log(13 / Re + K / 3.7)) ^ (-2)
    #
    return Lambda_dp

def k_lin(L_m, DN_mm, Lambda_dp):
    k_lin = L_m * Lambda_dp * 1000 / DN_mm
    return k_lin

def k_coude(Din_mm):
    if(Din_mm > 50):
            k_coude = 0.3
            
    elif(Din_mm<= 50):
        k_coude = 0.5
    
    elif(Din_mm<= 30):
        k_coude = 1
    
    elif(Din_mm<= 12):
        k_coude = 1.5 
    else:
        k_coude = 1  
    return k_coude

def dP_mbar(m_gs, L_m, DN_mm ,Elbow_nb, gas, P_bar, T_K ):
    Rho = PropsSI("D", "P", P_bar * 100000, "T", T_K, "HEOS::" + gas)
    mu = PropsSI("viscosity", "P", P_bar * 100000, "T", T_K, "HEOS::" + gas)
    PI = math.pi

    V = (m_gs / Rho / 1000) / (PI * (DN_mm / 1000) ** 2 / 4)
    Re = Re_(DN_mm, V, Rho, mu)
    Lambda = Lambda_dp(DN_mm, Re)

    K_tot = k_lin(L_m, DN_mm, Lambda) + k_coude(DN_mm) * Elbow_nb
    dP_mbar = K_tot * V ** 2 * Rho / 2 * 10 ** -2
    # if dP_mbar > 1000:
    #     return f"{dP_mbar*u('mbar'):,g}" 
    # elif:
    #     return f"{dP_mbar*u('mbar'):,.2f}"
    return f"{dP_mbar*u('mbar'):,.2f}"
